In [ ]:
# vinai/PhoGPT-4B-Chat-v0.1

In [ ]:
# !git clone https://huggingface.co/vinai/PhoGPT-4B-Chat-v0.1

Cloning into 'PhoGPT-4B-Chat-v0.1'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 43 (delta 12), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (43/43), 265.51 KiB | 494.00 KiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/Colab Notebooks/PhoGPT-4B-Chat-v0.1/.git/hooks/post-checkout': Permission denied
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model.bin

See: `git lfs help smudge` for more details.


In [ ]:
!pip install -q transformers einops torch accelerate

Model: vinai/PhoGPT-4B-Chat-v0.1

* Type: Instruction following & Chat
* Model Size: 3.7B
* Context length: 8192
* Vocab size: 20K
* Training data size: 70K instructional prompt and response pairs & 290K conversations
* Note: PROMPT_TEMPLATE = "### Câu hỏi: {instruction}\n### Trả lời:"

In [ ]:
# coding: utf8
import torch, accelerate,transformers, einops
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

# model_path = "PhoGPT-4B-Chat-v0.1"
model_path = "PhoGPT-4B-Chat-v01"

config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
config.init_device = "cuda"

model = AutoModelForCausalLM.from_pretrained(model_path, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True, use_auth_token=None)
# If your GPU does not support bfloat16:
# model = AutoModelForCausalLM.from_pretrained(model_path, config=config, torch_dtype=torch.float16, trust_remote_code=True)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

PROMPT_TEMPLATE = "### Câu hỏi: {instruction}\n### Trả lời:"

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
def model_generator(model, instruction):
  input_prompt = PROMPT_TEMPLATE.format_map({"instruction": instruction})
  input_ids = tokenizer(input_prompt, return_tensors="pt")

  outputs = model.generate(
  inputs=input_ids["input_ids"].to("cuda"),
  # attention_mask=input_ids["attention_mask"].to("cuda"),
  do_sample=True,
  temperature=1.0,
  top_k=50,
  top_p=0.9,
  max_new_tokens=1024,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
  # return_dict_in_generate=True, output_scores=True
  # output_hidden_states = True,
  )

  response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
  response = response.split("### Trả lời:")[1]
  return response

In [ ]:
# Some instruction examples
# instruction = "Viết bài văn nghị luận xã hội về {topic}"
# instruction = "Viết bản mô tả công việc cho vị trí {job_title}"
# instruction = "Sửa lỗi chính tả:\n{sentence_or_paragraph}"
# instruction = "Dựa vào văn bản sau đây:\n{text}\nHãy trả lời câu hỏi: {question}"
# instruction = "Tóm tắt văn bản:\n{text}"

# instruction = "Viết bài văn nghị luận xã hội về an toàn giao thông"
# instruction = "Sửa lỗi chính tả:\nTriệt phá băng nhóm kướp ô tô, sử dụng \"vũ khí nóng\""

# instruction = "Hãy đưa ra 10 ý tưởng về marketing bán hàng và các bước để thực hiện những ý tưởng đó"

In [ ]:
# model_path = "PhoGPT-4B-Chat-v0.1"
instruction = "em tên là gì? em nhà ở đâu thế? sở thích của em là gì?"
print(model_generator(model, instruction))

Em tên là Anna, và em sống ở thành phố New York. Còn sở thích của em là đọc sách, xem phim và chơi bóng rổ.
